In [20]:
import os 
import requests

In [148]:
#Create spotify class which contains all functions and attributes related to spotify API
class SpotifyAPI:
    
    authorize_url = 'https://accounts.spotify.com/authorize'
    tracks_request_url = 'https://api.spotify.com/v1/me/tracks'
    
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_authorization_code(self):
        
        client_id = self.client_id
        client_secret = self.client_secret
        authorize_url = self.authorize_url
        
        authorize_params = {
            'client_id':client_id,
            'response_type':'token',
            'redirect_uri':'https://developer.spotify.com/documentation/web-api/reference-beta/#endpoint-get-users-saved-tracks',
            'scope':'user-library-read'
        }
        
        r = requests.get(authorize_url, params=authorize_params)
        
        print(r.url)
    
    def get_track_data(self):
        
        access_token = self.get_access_token()
        
        tracks_request_url = self.tracks_request_url
        
        tracks_request_header = {
            'Authorization': f'Bearer {access_token}',
            'scope':'user-library-read'
        }
        
        r = requests.get(tracks_request_url, headers=tracks_request_header)
        
        return r.status_code

In [149]:
#Create spotify client object with spotify app client id and secret key
os.environ['SPOTIFY_API_CLIENT_ID'] = '853fe527a1704e18845533fb15278ead'
os.environ['SPOTIFY_API_SECRET_KEY'] = 'fc34081faaa34d27ab794e1aef9d1026'

spotify_client = SpotifyAPI(os.environ['SPOTIFY_API_CLIENT_ID'], os.environ['SPOTIFY_API_SECRET_KEY'])

In [150]:
#Get auth code url 
auth_code_url = spotify_client.get_authorization_code()

https://accounts.spotify.com/login?continue=https%3A%2F%2Faccounts.spotify.com%2Fauthorize%3Fscope%3Duser-library-read%26response_type%3Dtoken%26redirect_uri%3Dhttps%253A%252F%252Fdeveloper.spotify.com%252Fdocumentation%252Fweb-api%252Freference-beta%252F%2523endpoint-get-users-saved-tracks%26client_id%3D853fe527a1704e18845533fb15278ead


In [151]:
#Token is taken from auth code url
token = 'BQAcaH4b5Tx6gr_NXukTHiCEj7VxVq3XhNzh1uEzWziO74y-nehIxkRm4y4CVEfHYPb6iMyyZ53DEQWVi5s33s5MVz7mhIeSnHSRvV3l60jWEw67OXDBXwE6p-JpAZyTVIiMHidRTQAjo0Z7xIaTNA'

#Get all track ids in spotify account
track_ids = []

for i in range(20):
    
    offset = i*50
    
    tracks_url = f'https://api.spotify.com/v1/me/tracks?offset={offset}&limit=50'

    header = {
        'Authorization': f'Bearer {token}'
    }

    tracks_response = requests.get(tracks_url, headers=header)
    
    tracks_json = tracks_response.json()
    
    for track in tracks_json['items']:
        track_id = track['track']['id']
        track_ids.append(track_id)

In [158]:
import pandas as pd

#Get all track features from spotify api
track_features_df = pd.DataFrame(columns=['danceability','energy','key','loudness','mode','speechiness','acousticness',
                                         'instrumentalness','liveness','valence','tempo'])

for track in track_ids:
    
    track_response = requests.get(f'https://api.spotify.com/v1/audio-features/{track}', headers=header)
    track_response_json = track_response.json()
    
    track_features = {
        'danceability':track_response_json['danceability'],
        'energy':track_response_json['energy'],
        'key':track_response_json['key'],
        'loudness':track_response_json['loudness'],
        'mode':track_response_json['mode'],
        'speechiness':track_response_json['speechiness'],
        'acousticness':track_response_json['acousticness'],
        'instrumentalness':track_response_json['instrumentalness'],
        'liveness':track_response_json['liveness'],
        'valence':track_response_json['valence'],
        'tempo':track_response_json['tempo']
    }
    
    track_features_df = track_features_df.append(track_features,ignore_index=True)